# HCE Predictive Policing Notebook

## Loading 

In [1]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.8.2 (default, Apr 27 2020, 15:53:34) 
[GCC 9.3.0]
Version info.
sys.version_info(major=3, minor=8, micro=2, releaselevel='final', serial=0)


In [3]:
import numpy as np
import sklearn as sk
import pandas as pd

## Reading in Data

In [4]:
data = pd.read_csv('compas_scores_two_years.csv')

In [5]:
data.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [6]:
data.race.value_counts()

African-American    3696
Caucasian           2454
Hispanic             637
Other                377
Asian                 32
Native American       18
Name: race, dtype: int64

SCRATCH SPACE

In [7]:
cut_df = data[['id', 'name','race','is_recid', 'two_year_recid']]
fp_fn = cut_df[cut_df.is_recid != cut_df.two_year_recid]
fp = cut_df[(cut_df.is_recid == 1)& (cut_df.two_year_recid==0)]

In [8]:
term1=len(cut_df[(cut_df.race=='African-American') & (cut_df.is_recid ==1)])/len(cut_df[(cut_df.race=='African-American') & (cut_df.two_year_recid ==0)])
term2=len(cut_df[(cut_df.race=='Caucasian') & (cut_df.is_recid ==1)])/len(cut_df[(cut_df.race=='Caucasian') & (cut_df.two_year_recid ==0)])

In [9]:
#len(data[(data.race==group1) & (data.is_recid ==1) & (data.two_year_recid ==0)])
g1_fp = len(cut_df[(cut_df.race=='African-American') & (cut_df.is_recid==1) & (cut_df.two_year_recid==0)])
#len(cut_df[(cut_df.race=='Caucasian') & (cut_df.is_recid==1) & (cut_df.two_year_recid==0)])
g1_tp = len(cut_df[(cut_df.race=='African-American') & (cut_df.is_recid==1)])
g1_tn = len(cut_df[(cut_df.race=='African-American') & (cut_df.is_recid==0) & (cut_df.two_year_recid==0)])

In [10]:
g1_tn

1660

In [11]:
g1_fp, g1_tp

(135, 2036)

Week of May 17:
Translating FRGS, FDR, FPR metrics into code and placing them in a jupyter notebook which will be the base of our project. We want these metrics to be attached to different stats papers and fully functional. Should take in the ProPublica data and output a figure. These metrics will be the backbone of the second part of the project, which talks about different ways to assess the fairness of the ProPublica/Northpointe debate, and brings in statistician’s own concepts of fairness


# Parity Metrics
Reference: https://stats.stackexchange.com/questions/336455/fpr-false-positive-rate-vs-fdr-false-discovery-rate

## FPGS Parity 
### Motivating Idea: What are your changes of being wrongly denied bail just given your race? 
AKA, Given you are of a certain group, what is the probability of your true label being different from your predicted label? We use this metric when we are trying to ensure predictive equity **for everyone without regard for actual outcome**.
$$ P(\hat{Y} = 1, {Y} = 0 | G) $$

We can also see this as **False Positive Group Selection** Parity. Aequitas describes this as examining the number of entities of the group with $\hat{Y} = 1$ and ${Y} = 0 $.

In [12]:
def fpgs_parity(data, group1, group2):
    '''
    Given two groups for which we'd like to compare the False Positive
    Group Selection parity, generate a ratio of the counts of Y_hat = 1
    and Y=0 for each group.
    '''
    # Group 1 Count Y_hat
    g1_yhat = len(data[(data.race==group1) & (data.is_recid ==1)])
    # Group 1 Count Y
    g1_y = len(data[(data.race==group1) & (data.two_year_recid ==0)])
    # Group 2 Count Y_hat
    g2_yhat = len(data[(data.race==group2) & (data.is_recid ==1)])
    # Group  2 Count Y                                                            
    g2_y = len(data[(data.race==group2) & (data.two_year_recid ==0)]) 
    
    return g1_yhat/g2_yhat, g1_y/g2_y

In [13]:
fpgs_parity(cut_df, 'African-American', 'Caucasian')
fpgs_parity(cut_df, 'Hispanic', 'Caucasian')

(0.23902439024390243, 0.2721774193548387)

## FDR Parity
### Motivating Idea: Among people denied bail, what are the chances you're innocent given your race? 
AKA, Given you are of a certain group and predicted to be denied bail, what is the probability that your true label is 0? We use this metric when trying to ensure predictive equity among people **for whom intervention is taken.** This means intervening for those denied bail to make sure they were predicted accurately. 
$$ P({Y} = 0 | G, \hat{Y} = 1) $$


We can also see this as **False Discovery Rate** Parity. Aequitas describes this as examining the false positives of a group within the predicted positives of the group.

In [14]:
def fdr_parity(data, group1, group2):
    '''
    Given two groups for which we'd like to compare the False Discovery 
    Rate parity, generate a ratio of the probabilities as specified above.
    
    We find this by calculating False Positives over Total Predicted Positives.  
    '''
    # Group 1 False Positive Count
    g1_false_predictions = len(data[(data.race==group1) & (data.is_recid ==1) & (data.two_year_recid ==0)])
    # Group 1 Total Positive Count
    g1_total_predictions = len(data[(data.race==group1) & (data.is_recid==1)])
    # Group 2 False Positive Count
    g2_false_predictions = len(data[(data.race==group2) & (data.is_recid ==1) & (data.two_year_recid ==0)])
    # Group 2 Total Positive Count
    g2_total_predictions = len(data[(data.race==group2) & (data.is_recid==1)])

    g1_fdr = g1_false_predictions/g1_total_predictions
    g2_fdr = g2_false_predictions/g2_total_predictions
    
    return g1_fdr, g2_fdr

In [15]:
fdr_parity(cut_df, 'African-American', 'Caucasian')

(0.06630648330058939, 0.0575609756097561)

## FPR Parity

### Motivating Idea: Among people who **should** be granted bail, what are the chances you were denied bail given your race?
AKA, Given your are of a certain group and your true label is 0, what is the probability that you were predicted to be denied bail? We use this metric when trying to ensure predictive equity for groups **that do not warrant intervention.** This means 
$$ P(\hat{Y} = 1 | G, {Y} = 0) $$


We can also see this as **False Positive Rate** Parity. Aequitas describes this as examining the fraction of false positives of a group within the predicted positive of the group.

In [16]:
def fpr_parity(data, group1, group2):
    '''
    Given two groups for which we'd like to compare the False Positive
    Rate parity, generate a ratio of the probabilities as specified above.
    
    We find this by calculating False Positives over False Positives and True Negatives. 
    '''
    # Group 1 False Positive Count
    g1_false_positives = len(data[(data.race==group1) & (data.is_recid ==1) & (data.two_year_recid ==0)])
    # Group 1 True Negative Count 
    g1_tn = len(data[(data.race==group1) & (data.is_recid==0) & (data.two_year_recid==0)])
    # Group 2 False Positive Count
    g2_false_positives= len(data[(data.race==group2) & (data.is_recid ==1) & (data.two_year_recid ==0)])
    # Group 2 True Negative Count
    g2_tn = len(data[(data.race==group2) & (data.is_recid==0) & (data.two_year_recid==0)])
    
    g1_fpr = g1_false_positives / (g1_false_positives+g1_tn)
    g2_fpr = g2_false_positives / (g2_false_positives+g2_tn)

    return g1_fpr, g2_fpr

In [17]:
fpr_parity(cut_df, 'African-American', 'Caucasian')

(0.07520891364902507, 0.0396505376344086)